# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [3]:
flights_df = pd.read_csv('flights.csv')
fuel_df = pd.read_csv('fuel_consumption.csv')
passengers_df= pd.read_csv('passengers.csv')

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

In [4]:
flights_df.head()

,Unnamed: 0,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,...,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name,flight_status,month,dep_time_format,average speed
0,0,2019-11-27,AA,AA_CODESHARE,AA,5052,OH,N557NN,5052,11057,...,NaN,NaN,NaN,NaN,NaN,NaN,0,11,16:28:00,160.645161
1,1,2019-12-26,AA,AA_CODESHARE,AA,5812,YV,N952LR,5812,13851,...,0.0,24.0,NaN,NaN,NaN,NaN,1,12,17:32:00,308.823529
2,2,2018-10-25,WN,WN,WN,1270,WN,N962WN,1270,15304,...,NaN,NaN,NaN,NaN,NaN,NaN,1,10,09:49:00,436.200000
3,3,2018-06-25,DL,DL,DL,2031,DL,N979AT,2031,10397,...,0.0,0.0,NaN,NaN,NaN,NaN,1,6,14:01:00,348.837209
4,4,2019-11-20,B6,B6,B6,2705,B6,N547JB,2705,11618,...,NaN,NaN,NaN,NaN,NaN,NaN,0,11,17:05:00,466.423358


In [5]:
flights_df.columns

Index(['Unnamed: 0', 'fl_date', 'mkt_unique_carrier', 'branded_code_share',
       'mkt_carrier', 'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'no_name', 'flight_status', 'month',
       'dep_time_format', 'average speed'],
      dtype='object')

In [218]:
# cancelled and diverted flights not considered delayed -- delete 
flights_df = flights_df.drop(['diverted','cancelled'],axis=1)

In [219]:
# A lot of delay data are missing (carrier_delay, weather_delay.. etc)
print("Percentage of valid data: ", 100-(flights_df.carrier_delay.isnull().sum()*100/len(flights_df)))
print("Percentage of missing data: ", flights_df.carrier_delay.isnull().sum()*100/len(flights_df) )

Percentage of valid data:  19.094202157051342
Percentage of missing data:  80.90579784294866


In [220]:
# Drop data on delays b/c too many missing data 
flights_df = flights_df.drop(['carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay'],axis=1)

In [221]:
# delete other ones that we do not need 
flights_df = flights_df.drop(['no_name','dup','cancellation_code','first_dep_time', 'total_add_gtime', 'longest_add_gtime','tail_num'],axis=1)

### Check for null values 

In [222]:
flights_df.isnull().sum()

fl_date                 0
mkt_unique_carrier      0
branded_code_share      0
mkt_carrier             0
mkt_carrier_fl_num      0
op_unique_carrier       0
op_carrier_fl_num       0
origin_airport_id       0
origin                  0
origin_city_name        0
dest_airport_id         0
dest                    0
dest_city_name          0
crs_dep_time            0
dep_time                0
dep_delay               0
taxi_out               44
wheels_off             44
wheels_on              44
taxi_in                44
crs_arr_time            0
arr_time                0
arr_delay               0
crs_elapsed_time        0
actual_elapsed_time     0
air_time               44
flights                 0
distance                0
flight_status           0
month                   0
dep_time_format         0
average speed          44
dtype: int64

Only 44 null values in 100,000 - can delete 

In [223]:
# delete flights where there is no airtime available 
flights_df = flights_df.dropna()

A lot of columns have duplicated information - drop

In [224]:
# drop duplicated info columns 
flights_df = flights_df.drop(['mkt_unique_carrier','branded_code_share','op_unique_carrier','mkt_carrier_fl_num'],axis=1)

In [225]:
flights_df = flights_df.drop(['flights','origin_city_name','dest_city_name'],axis=1)

### Change date to corresponding month and day 

In [226]:
flights_df['day'] = pd.DatetimeIndex(flights_df['fl_date']).day

In [ ]:
import calendar
flights_df['month'] = flights_df['month'].apply(lambda x: calendar.month_abbr[x])

In [227]:
flights_df['fl_date'] = pd.to_datetime(flights_df['fl_date'])
flights_df['weekday'] = flights_df['fl_date'].dt.dayofweek

In [228]:
# Since i have month and date on separate column, delete fl_date
flights_df = flights_df.drop(['dest_airport_id','origin_airport_id'],axis=1)

Taxi in and out values will not be given in test dataset - drop

In [229]:
flights_df = flights_df.drop(['taxi_out', 'wheels_off', 'wheels_on', 'taxi_in'],axis=1)

### Change carrier names to something easier to understand

In [231]:
#change carrier names 
flights_df['mkt_carrier'].replace({
    'UA':'United Airlines',
    'AS':'Alaska Airlines',
    '9E':'Endeavor Air',
    'B6':'JetBlue Airways',
    'EV':'ExpressJet',
    'F9':'Frontier Airlines',
    'G4':'Allegiant Air',
    'HA':'Hawaiian Airlines',
    'MQ':'Envoy Air',
    'NK':'Spirit Airlines',
    'OH':'PSA Airlines',
    'OO':'SkyWest Airlines',
    'VX':'Virgin America',
    'WN':'Southwest Airlines',
    'YV':'Mesa Airline',
    'YX':'Republic Airways',
    'AA':'American Airlines',
    'DL':'Delta Airlines',
    'G7': 'GoJet Airlines',
    'QX': 'Horizin Air',
    'AX': 'Trans States Airlines', 
    'C5': 'CommutAir',
    'PT': 'Piedmont Airlines',
    'ZW': 'Air Wisconsin',
    'CP': 'Compass Airlines',
    'EM': 'Empire Airlines',
    '9K':'Cape Air',
    'KS':'Peninsula Airways'
}, inplace=True)

In [232]:
flights_df.mkt_carrier.unique()

array(['American Airlines', 'Southwest Airlines', 'Delta Airlines',
       'JetBlue Airways', 'United Airlines', 'Hawaiian Airlines',
       'Spirit Airlines', 'Alaska Airlines', 'Virgin America',
       'Allegiant Air', 'Frontier Airlines'], dtype=object)

In [233]:
flights_df.flight_status.value_counts(normalize=True)

0    0.631375
1    0.368625
Name: flight_status, dtype: float64

This means that 62.48% of the flights have no delays, but that can be that they arrived either early or on time, and 37.52% of the flight were delayed. Quite a significant number if you think about it.

### Change departure times to hours 

In [236]:
flights_df['dep_time_format'] = pd.to_datetime(flights_df['dep_time_format'])
flights_df['dep_time_format'] = flights_df['dep_time_format'].dt.hour


In [243]:
#function to change integer time to standard time in string
def time_row(row):
    row_int = int(row)
    row_str = str(row_int)
    len_row = len(row_str)
    if len_row == 1:
        minute = '00'
        hour = row_str
        row = hour + ':' + minute
    if len_row == 2:
        minute =  '0'+ row_str[1]
        hour = row_str[0]
        row = hour + ':' + minute
    if len_row == 3:
        minute = row_str[1]+row_str[2]
        hour = row_str[0]
        row = hour + ':' + minute
    elif len_row == 4:
        minute = row_str[2] + row_str[3]
        hour = row_str[0] + row_str[1]
        row = hour + ':' + minute
        if row == '24:00':
            row = '23:59'
    return str(row)

In [244]:
#applyting the function and changing to format to date and time
flights_df['dep_time_format'] = flights_df['dep_time'].apply(time_row)

flights_df['dep_time_format'] = pd.to_datetime(\
                                                       flights_df['dep_time_format'],\
                                                       format='%H:%M').dt.time



In [245]:
def hr_func(ts):
    return ts.hour

flights_df['dep_hour'] = flights_df['dep_time_format'].apply(hr_func)


In [250]:
flights_df.arr_delay.describe()

count    97682.000000
mean         4.039936
std         37.970272
min        -85.000000
25%        -15.000000
50%         -6.000000
75%          8.000000
max        349.000000
Name: arr_delay, dtype: float64

In [9]:

#encoding function to numerical describe the duration of a flight based on three intervals (less than 3 hours, in between 3-6 hours and greater then 6 hours)
def time_cat(flights_df, col):
    '''Determine if flight length is SHORT (0), MEDIUM(1) or LONG(2) based on expected elapsed flight time. '''
    length=[]
    for i in flights_df[col]: 
        if (i >=-85) and i <= -15: 
            length.append(0) # 0 = no delay
        elif (i >-15) and (i <= -0): 
            length.append(1) # 1 = no delay 
        elif (i > 0) and (i<= 8):
            length.append(2) # 2 = short delay 
        elif (i > 8) and (i<= 350):
            length.append(3) # 3 = medium to long delay 

    flights_df['time_cat'] = length

In [283]:
time_cat(flights_df,'arr_delay')

##### Creating column with average delay for each bin 

In [284]:
flights_df.loc[(flights_df['time_cat']== 0),'avg_delay'] = flights_df.loc[(flights_df['time_cat']== 0), 'arr_delay'].mean()
flights_df.loc[(flights_df['time_cat']== 1),'avg_delay'] = flights_df.loc[(flights_df['time_cat']== 1), 'arr_delay'].mean()
flights_df.loc[(flights_df['time_cat']== 2),'avg_delay'] = flights_df.loc[(flights_df['time_cat']== 2), 'arr_delay'].mean()
flights_df.loc[(flights_df['time_cat']== 3),'avg_delay'] = flights_df.loc[(flights_df['time_cat']== 3), 'arr_delay'].mean()
flights_df.loc[(flights_df['time_cat']== 4),'avg_delay'] = flights_df.loc[(flights_df['time_cat']== 4), 'arr_delay'].mean()

### Dropping irrelevant columns 

In [237]:
flights_df = flights_df.drop(['average speed'],axis=1)
flights_df = flights_df.drop(['op_carrier_fl_num'],axis=1)
flights_df = flights_df.drop(['fl_date'],axis=1)

In [266]:
flights_df = flights_df.drop(['flight_status'],axis=1)

KeyError: "['flight_status'] not found in axis"

### Change object values to categorical values 

In [240]:
for col in ['mkt_carrier','origin','dest','month','weekday']:
    flights_df[col]=flights_df[col].astype('category')

In [242]:
flights_df["mkt_carrier"] = flights_df["mkt_carrier"].cat.codes
flights_df["origin"] = flights_df["origin"].cat.codes
flights_df["dest"] = flights_df["dest"].cat.codes

In [246]:
flights_df = flights_df.drop(['crs_arr_time','dep_time','dep_time_format'],axis=1)

In [247]:
flights_df = flights_df.drop(['crs_dep_time'],axis=1)

In [248]:
flights_df = flights_df.drop(['arr_time'],axis=1)

In [249]:
flights_df.head()

,mkt_carrier,origin,dest,dep_delay,arr_delay,crs_elapsed_time,actual_elapsed_time,air_time,distance,flight_status,month,weekday,dep_hour
0,2,74,145,-6.0,-4.0,59,61.0,31.0,83,0,11,2,16
1,2,249,95,40.0,24.0,70,54.0,34.0,175,1,12,3,17
2,7,349,316,-6.0,0.0,120,126.0,100.0,727,1,10,3,9
3,3,22,363,19.0,31.0,71,83.0,43.0,250,1,6,0,14
4,6,118,127,19.0,-10.0,186,157.0,137.0,1065,0,11,2,17


In [290]:
flights_df.to_csv(r'flights_final.csv')